In [1]:
import netCDF4 as nc
import numpy as np
import geopandas as gpd
import xarray as xr
import matplotlib.pyplot as plt
from shapely.geometry import box
from rasterio.mask import mask

In [ ]:
#reading all downloaded temperature files into a single variable
temp = xr.open_mfdataset('D:/Temperature-Air-2m-Max-Day-Time_C3S*.nc', combine='nested', concat_dim = 'time')

C:\Users\ASHWATHY\AppData\Roaming\Python\Python312\site-packages\xarray\backends\plugins.py:110: RuntimeWarning: Engine 'cfgrib' loading failed:
Cannot find the ecCodes library
  external_backend_entrypoints = backends_dict_from_pkg(entrypoints_unique)


In [ ]:
#reading all downloaded humidity files into a single variable
Humidity = xr.open_mfdataset('D:/Relative-Humidity-2m-15h_C3S*.nc', combine = 'nested', concat_dim = 'time')

In [ ]:
#shapefile of kerala
kerala_gdf = gpd.read_file('D:/kerala_st_bound.shp')

In [5]:
minx, miny, maxx, maxy = kerala_gdf.total_bounds
bbox = box(minx, miny, maxx, maxy)                  #bounding box min-max

clipped_data_temp = temp.sel(lon=slice(minx, maxx), lat=slice(miny, maxy))       #clipping our data
#clipped_data_humidity = Humidity_23.sel(lon=slice(minx, maxx), lat=slice(miny, maxy))

In [ ]:
masked_data_temp = temp.where((temp.lon >= minx) & (temp.lon <= maxx) &        #again strictly filtering our data within bbox
                       (temp.lat >= miny) & (temp.lat <= maxy), drop=True)

masked_data_humidity = Humidity.where((Humidity.lon >= minx) & (Humidity.lon <= maxx) & 
                       (Humidity.lat >= miny) & (Humidity.lat <= maxy), drop=True)

'masked_data_humidity = Humidity_23.where((Humidity_23.lon >= minx) & (Humidity_23.lon <= maxx) & \n                       (Humidity_23.lat >= miny) & (Humidity_23.lat <= maxy), drop=True)'

In [ ]:
masked_data_temp.to_netcdf('D:/kerala_max_temp.nc')
masked_data_humidity.to_netcdf('D:/kerala_humidity.nc')

In [ ]:
# combine all the temp datasets into a single nc file and a single nc file for humidity too
temp_12_yrs = xr.open_mfdataset('D:/kerala_max_temp*.nc', combine='nested', concat_dim = 'time')
humid_12_yrs = xr.open_mfdataset('D:/kerala_humidity*.nc', combine='nested', concat_dim = 'time')

"\n# combine all the temp datasets into a single nc file and a single nc file for humidity too\ntemp_10_yrs = xr.open_mfdataset('D:/DUK/EWYL/Yr_wise_temp_nc/kerala_max_temp*.nc', combine='nested', concat_dim = 'time')\nhumid_10_yrs = xr.open_mfdataset('D:/DUK/EWYL/Yr_wise_humid_nc/kerala_humidity*.nc', combine='nested', concat_dim = 'time')\n"

In [ ]:
# save the 10 yr temperature and humidity data as 2 different nc files
temp_12_yrs.to_netcdf('D:/temp_12_yrs.nc')
humid_12_yrs.to_netcdf('D:/humidity_12_yrs.nc')

"\n# save the 10 yr temperature and humidity data as 2 different nc files\ntemp_10_yrs.to_netcdf('D:/DUK/EWYL/temp_10_yrs.nc')\nhumidity_10_yrs.to_netcdf('D:/DUK/EWYL/humidity_10_yrs.nc')\n"

In [ ]:
#reading temp and humidity files to combine and calculate THI
max_temp = xr.open_dataset('D:/temp_12_yrs.nc')
humid = xr.open_dataset('D:/humidity_12_yrs.nc')

"\nmax_temp = xr.open_dataset('D:/DUK/EWYL/temp_10_yrs.nc')\nhumid = xr.open_dataset('D:/DUK/EWYL/humidity_10_yrs.nc')\n"

In [17]:
# convert Kelvin to Fahrenheit
max_temp['Temperature_Air_2m_Max_Day_Time'] = ((max_temp['Temperature_Air_2m_Max_Day_Time'] - 273.15)*(9/5)) + 32

In [18]:
#convert humidity in % to fraction
humid['Relative_Humidity_2m_15h'] = humid['Relative_Humidity_2m_15h']/100

In [ ]:
# combine temperature & humidity data into a single variable 
combined_data = xr.merge([max_temp,humid])

In [ ]:
# save it as a netCDF file
combined_data.to_netcdf('D:/temp_humid.nc')

In [21]:
# Read the netCDF file and calculate the THI value
combined_data['THI'] = combined_data['Temperature_Air_2m_Max_Day_Time'] - (0.55 - 0.55 * combined_data['Relative_Humidity_2m_15h']) * (combined_data['Temperature_Air_2m_Max_Day_Time'] - 58)

In [22]:
# assign the data to a variable called THI_data
THI_data = combined_data['THI']

In [ ]:
# save the data as a netCDF file
THI_data.to_netcdf('D:/THI_12_yrs.nc')